In [1]:
from keras.datasets import mnist

(train_X, train_y), (test_X, test_y) = mnist.load_data()

print('X_train: ' + str(train_X.shape))
print('Y_train: ' + str(train_y.shape))
print('X_test:  '  + str(test_X.shape))
print('Y_test:  '  + str(test_y.shape))

X_train: (60000, 28, 28)
Y_train: (60000,)
X_test:  (10000, 28, 28)
Y_test:  (10000,)


In [8]:
import time
import numpy as np
import random as random
from scipy.special import softmax
import math
import pandas as pd
from matplotlib import pyplot as plt

class Network():
    
    # Utils
    
    def __generate_random_matrix(self, shape_x, shape_y, max_value=1):
        return np.random.rand(shape_x, shape_y) * 2 * max_value - max_value
    
    def __generate_random_number(self, max_value=0.1):
        return random.random() * 2 * max_value - max_value
    
    # Constructor
    
    def __init__(self, layer_shapes, function, max_value=1):
        self.weights = []
        self.biases = []
        self.function = function
        self.all_indexes = []
        
        for layer_shape in layer_shapes:
            shape_x, shape_y = layer_shape
            self.weights.append(self.__generate_random_matrix(shape_x, shape_y, max_value=max_value))
            self.biases.append(self.__generate_random_number())
            
        self.map_size = 25
        
        self.filter_weights = []
        for i in range(self.map_size):
            var = 2.0 / 4
            w = np.random.normal(0, var, size=(4, 4))
            self.filter_weights.append(w)
        
    # Code
    
    @staticmethod
    def normalize(X):
        return X / 254
    
    @staticmethod
    def get_result(prediction):
        return np.where(prediction == np.max(prediction))[0][0]
    
    def __relu(self, z, deriv=False):
        if deriv:
            return 1 / (1 + np.exp(-z / np.max(z))) if np.mean(z) > 10 else 1 / (1 + np.exp(-z))
        else:
            return np.log(1 + np.exp(z / np.max(z))) if np.mean(z) > 10 else np.log(1 + np.exp(z))
        
    def __activation_function(self, z, deriv=False):
        return self.__relu(z, deriv)
    
    def __softmax_derivative(self, x):
        return softmax(x) * (1 - softmax(x))
    
    def convolution(self, X):
        
        frames = []
        
        # Iteracja 25x - liczba cech
        for filter_weight in self.filter_weights:
            frame = []
            # Przechodzenie filtra 4x4 po obrazie 28x28
            for i in range(24):
                for j in range(24):
                    # Operacja konwolucji
                    frame.append(np.sum(X[i:i+4, j:j+4] * filter_weight))
            # Pobudzenie w warstwie konwolucji
            frames.append(self.__activation_function(np.array(frame).reshape(24,24)))
            
        return frames
                
    def pooling(self, X):
        
        new_frames = []
        indexes = []
        
        # Iteracja 25x - liczba cech
        for frame in X:
            iterations = 12
            new_frame = []
            # Przechodzenienie filtra 2x2 po macierzy 24x24
            for i in range(iterations):
                for j in range(iterations):
                    # Operacja max pooling
                    new_frame.append(np.amax(frame[i*2:i*2+1, j*2:j*2+1]))
            new_frames.append(np.array(new_frame))
            
        return np.array(new_frames, dtype=np.float128).reshape((1,3600))
        
    
    def forward(self, X, predict=False):
        
        # Convolution
        X = self.convolution(X)
        # -----------
        
        # Max Pooling
        X = self.pooling(X)
        # -----------
        
        aa = [X]
        zz = [X]
        a = X
        
        for i, (w, b) in enumerate(zip(self.weights, self.biases)):
            
            z = a @ w + b
            
            if i == len(self.weights) - 1:
                a = softmax(z, axis=1)
            else:
                a = self.__activation_function(z)
                
            aa.append(a)
            zz.append(z)
            
        if predict:
            return a
        else:
            return aa, zz
        
    def train(self, X, y, learning_rate, max_epochs, max_error):
        
        costs = []
        epochs = 0
        
        for i in range(max_epochs):
            for j, X_matrix in enumerate(X):
                
                # Forward
                aa, zz = self.forward(X_matrix)
                # -------
                
                # Backpropagation
                # ...
                # ---------------
                
                print(f'Progress | Epoch: {i}, Iteration: {j}', end='\r')
                
            epochs += 1
            
        print(f'\nTraining completed')
        return epochs

In [9]:
X = []
for X_matrix in train_X[0:100]:
    X.append(Network.normalize(X_matrix))
X = np.array(X, dtype=np.float128)
    
labels = []
for y in train_y:
    label = np.zeros(10)
    label[y] = 1
    labels.append(label)
labels = np.array(labels, dtype=np.float128)

X_test = []
for X_matrix in test_X[0:100]:
    X_test.append(Network.normalize(X_matrix))
X_test = np.array(X_test, dtype=np.float128)

In [10]:
np.random.seed(243)
random.seed(243)

layer_shapes=[(3600, 15), (15, 10)]

network = Network(layer_shapes=layer_shapes, function='sigmoid')

epoch = network.train(X, 
              labels, 
              learning_rate=0.1, 
              max_epochs=10,
              max_error=0.05)

sum = 0
for X_, label_i in zip(X_test, test_y[0:100]):
    z = network.forward(X_, predict=True)
    if Network.get_result(z) == label_i:
        sum += 1

accs = round(sum / len(X_test) * 100, 2)

print(f'Accuracy: {accs}%')

Progress | Epoch: 9, Iteration: 99
Training completed
Accuracy: 8.0%
